In [61]:
import sys
import os

# خلي Python يشوف مجلد src
sys.path.append(os.path.abspath(os.path.join("..", "src")))
# دلوقتي تقدر تستدعي الموديولز
from features.distances import add_distance_features
from features.timeparts import add_time_features
from features.encoding import add_one_hot_encoding
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [62]:
df = pd.read_csv(r"G:\ML mostafa saad\slides\my work\13 Project 1 - Regression - Trip Duration Prediction\data\raw\train.csv")

print(df.shape)
print(df.head())
print(df.info())

(1458644, 11)
          id  vendor_id      pickup_datetime     dropoff_datetime  \
0  id2875421          2  2016-03-14 17:24:55  2016-03-14 17:32:30   
1  id2377394          1  2016-06-12 00:43:35  2016-06-12 00:54:38   
2  id3858529          2  2016-01-19 11:35:24  2016-01-19 12:10:48   
3  id3504673          2  2016-04-06 19:32:31  2016-04-06 19:39:40   
4  id2181028          2  2016-03-26 13:30:55  2016-03-26 13:38:10   

   passenger_count  pickup_longitude  pickup_latitude  dropoff_longitude  \
0                1        -73.982155        40.767937         -73.964630   
1                1        -73.980415        40.738564         -73.999481   
2                1        -73.979027        40.763939         -74.005333   
3                1        -74.010040        40.719971         -74.012268   
4                1        -73.973053        40.793209         -73.972923   

   dropoff_latitude store_and_fwd_flag  trip_duration  
0         40.765602                  N            455  
1 

دلوقتي خدنا فكره عامه عن الداتا بتاعتنا لقيناها مش معبره كفايه ولكن منها نقدر ننشأ features جديده تكون معبره اكتر وليها علاقه مباشره بالتارجت بتاعنا 
زي ايه بقا نجيب بعض المسافات من معلومات خطوط االطول ودواير العرض بتاع مكان الاقلاع ومكان الانتهاء 
Distances: Haversine, Manhattan
Directions: Bearing ,speed_kmh
وبرضو عمود الداتاتايم لو فصلناه يبقا افضل لان الساعه ممكن تكون اهم من التاريخ وهل اليوم ده كان ويك اند ولا عادي فبرضو هنعمل شغل علي اعمدة الداتا تايم 
كل ده هنستخدم ملف  ال features لان كل الدوال منظمه جواه

In [63]:
# ==== 9️⃣ Feature Ideas Generation ====

# 1. Distance-based features
from features.distances import add_distance_features
from features.timeparts import add_time_features

# Make sure df is defined
df = pd.read_csv(r"G:\ML mostafa saad\slides\my work\13 Project 1 - Regression - Trip Duration Prediction\data\raw\train.csv")
df['trip_duration_min'] = df['trip_duration'] / 60

df = add_distance_features(df)

# 2. Time-based features (using pickup_datetime)
df = add_time_features(df, datetime_col="pickup_datetime")

print("\n Columns after add_distance_features:")
print(df.columns.tolist())

# 3. Quick check
print("\n✅ Features added successfully!")
print(df.info())
print(df.head(10)[[
    "trip_duration_min", 
    "haversine_km", "manhattan_km", "bearing_deg",
    "hour", "weekday", "month", "is_weekend", "is_rush_hour"
]])
print(df[["trip_duration_min", "haversine_km", "manhattan_km", "speed_kmh", "bearing_deg"]].head())

✅ add_distance_features: 1458644/1458644 rows got valid speed_kmh

 Columns after add_distance_features:
['id', 'vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'store_and_fwd_flag', 'trip_duration', 'trip_duration_min', 'haversine_km', 'manhattan_km', 'bearing_deg', 'speed_kmh', 'hour', 'weekday', 'month', 'is_weekend', 'is_rush_hour']

✅ Features added successfully!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 21 columns):
 #   Column              Non-Null Count    Dtype         
---  ------              --------------    -----         
 0   id                  1458644 non-null  object        
 1   vendor_id           1458644 non-null  int64         
 2   pickup_datetime     1458644 non-null  datetime64[ns]
 3   dropoff_datetime    1458644 non-null  object        
 4   passenger_count     1458644 non-null  int64         
 5   pic

الداتا عندنا بتاع التارجت فيها شوية  skewed 
Log Transformationيعني التوزيع بتاعها مش طبيعي اوي عشان كده رأيي نطبق عليه دالة ال 

In [64]:
df = df.drop(columns=["trip_duration"])


In [65]:
df["log_trip_duration_min"] = np.log1p(df["trip_duration_min"])
print (df.head(10))

          id  vendor_id     pickup_datetime     dropoff_datetime  \
0  id2875421          2 2016-03-14 17:24:55  2016-03-14 17:32:30   
1  id2377394          1 2016-06-12 00:43:35  2016-06-12 00:54:38   
2  id3858529          2 2016-01-19 11:35:24  2016-01-19 12:10:48   
3  id3504673          2 2016-04-06 19:32:31  2016-04-06 19:39:40   
4  id2181028          2 2016-03-26 13:30:55  2016-03-26 13:38:10   
5  id0801584          2 2016-01-30 22:01:40  2016-01-30 22:09:03   
6  id1813257          1 2016-06-17 22:34:59  2016-06-17 22:40:40   
7  id1324603          2 2016-05-21 07:54:58  2016-05-21 08:20:49   
8  id1301050          1 2016-05-27 23:12:23  2016-05-27 23:16:38   
9  id0012891          2 2016-03-10 21:45:01  2016-03-10 22:05:26   

   passenger_count  pickup_longitude  pickup_latitude  dropoff_longitude  \
0                1        -73.982155        40.767937         -73.964630   
1                1        -73.980415        40.738564         -73.999481   
2                1     

عندنا أعمدة في الداتا مش أرقام، او ارقام مترتبه
:
- `vendor_id`
- `store_and_fwd_flag`
- `hour`
- `weekday`
- `month`

الموديل ما بيعرفش يتعامل مع الحروف أو القيم اللي مش أرقام.  
ولو أرقام عادية (1، 2، 3 …) الموديل هيفتكر إن في ترتيب ومعنى للفرق بينهم، وده مش صح.

عشان كده بنستخدم **One-Hot Encoding**:
- بيحوّل كل قيمة كاتيجوري لعمود جديد فيه 0 و 1.
- بيمنع الموديل يفتكر إن في علاقة ترتيب بين القيم.
- بيسهّل على الموديل يتعلم صح.
- 
مثال:  
لو عندنا `vendor_id` (1 أو 2) → بعد الـ encoding هيتحوّل لعمود `vendor_id_2` فيه:
- 1 لو الفيندور = 2
- 0 لو الفيندور = 1  

وبكده نكون مثلنا الكاتيجوري بطريقة أوضح للموديل.

In [66]:
df_encodedd = add_one_hot_encoding(df)
print(df_encodedd.columns.tolist())

print(df_encodedd.head(10))

['id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude', 'trip_duration_min', 'haversine_km', 'manhattan_km', 'bearing_deg', 'speed_kmh', 'hour', 'is_weekend', 'is_rush_hour', 'log_trip_duration_min', 'vendor_id_2', 'store_and_fwd_flag_Y', 'weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6']
          id     pickup_datetime     dropoff_datetime  passenger_count  \
0  id2875421 2016-03-14 17:24:55  2016-03-14 17:32:30                1   
1  id2377394 2016-06-12 00:43:35  2016-06-12 00:54:38                1   
2  id3858529 2016-01-19 11:35:24  2016-01-19 12:10:48                1   
3  id3504673 2016-04-06 19:32:31  2016-04-06 19:39:40                1   
4  id2181028 2016-03-26 13:30:55  2016-03-26 13:38:10                1   
5  id0801584 2016-01-30 22:01:40  2016-01-30 22:09:03                6   
6  id1813257 2016-06-

In [67]:
print(df_encodedd.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1458644 entries, 0 to 1458643
Data columns (total 30 columns):
 #   Column                 Non-Null Count    Dtype         
---  ------                 --------------    -----         
 0   id                     1458644 non-null  object        
 1   pickup_datetime        1458644 non-null  datetime64[ns]
 2   dropoff_datetime       1458644 non-null  object        
 3   passenger_count        1458644 non-null  int64         
 4   pickup_longitude       1458644 non-null  float64       
 5   pickup_latitude        1458644 non-null  float64       
 6   dropoff_longitude      1458644 non-null  float64       
 7   dropoff_latitude       1458644 non-null  float64       
 8   trip_duration_min      1458644 non-null  float64       
 9   haversine_km           1458644 non-null  float64       
 10  manhattan_km           1458644 non-null  float64       
 11  bearing_deg            1458644 non-null  float64       
 12  speed_kmh              14586